# LinkedIn Hiring Rate Analysis: Labor Market Dynamics (2018-2025)
The analysis of LinkedIn Hiring Rate (LHR) data reveals distinct patterns across the studied countries. 

In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Tabs, TabPanel, Toggle, CustomJS
from bokeh.palettes import Category20
from bokeh.layouts import column
import openpyxl
import statsmodels.formula.api as smf

In [2]:
# import employment data API_SL.EMP.TOTL.SP.ZS_DS2_en_csv_v2_14479

employment_data = pd.read_csv('../../data/WDI/API_SL.EMP.TOTL.SP.ZS_DS2_en_csv_v2_14479.csv', skiprows=4)

# filter by country (in this case, Algeria, Morrocco, United States))
countries = ['Algeria', 'Morocco', 'United States']
filtered_employment_data = employment_data[employment_data['Country Name'].isin(countries)]

employment_pivot = filtered_employment_data.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
    var_name='Year',
    value_name='Value'
)
# Filter out non-numeric years before converting
employment_pivot = employment_pivot[employment_pivot['Year'].str.isnumeric()]

# Convert Year to numeric just in case
employment_pivot['Year'] = employment_pivot['Year'].astype(int)

# filter years from 2016 to 2024
employment_pivot = employment_pivot[employment_pivot['Year'].astype(int).between(2016, 2024)]

# Sort by year to ensure correct order
employment_pivot = employment_pivot.sort_values('Year')

# Compute year-to-year change in percentage points
employment_pivot['Delta_EPR_pp'] = (
    employment_pivot.groupby('Country Name')['Value'].diff()
)

# (Optional) Also compute % growth relative to previous year
employment_pivot['EPR_Growth_pct'] = (
    employment_pivot.groupby('Country Name')['Value'].pct_change() * 100
)

# Keep only relevant years (2016–2024 already filtered)
employment_pivot = employment_pivot.reset_index(drop=True)



In [3]:
def clean_linkedin_data(excel_file):
    """Clean and prepare LinkedIn data from Excel file."""
    try:
        data_corrected = pd.read_excel(excel_file, sheet_name="2A - LHR by Ctry", header=3)

        data_cleaned = data_corrected.rename(columns={
            'Unnamed: 1': 'Month',
            'Unnamed: 2': 'Country',
            'Unnamed: 3': 'LHR (YOY)'
        })

        data_cleaned = data_cleaned.dropna(subset=['Month', 'Country', 'LHR (YOY)'])
        data_cleaned = data_cleaned.iloc[1:]
        data_cleaned = data_cleaned.drop(columns=['Unnamed: 0'])

        # Clean up whitespace and standardize country names
        data_cleaned["Country"] = data_cleaned["Country"].str.strip()
        data_cleaned["Country"] = data_cleaned["Country"].replace({
            "Turkey": "Turkiye",
            "Türkiye": "Turkiye"
        })

        return data_cleaned

    except Exception as e:
        print(f"Error during data cleaning: {str(e)}")
        return None



In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (
    ColumnDataSource, HoverTool, Tabs, TabPanel, Toggle, CustomJS, Button
)
from bokeh.layouts import column
from bokeh.palettes import Category20
import pandas as pd

def create_country_plots(excel_file):
    """Create interactive plots showing LinkedIn Hiring Rate by Country with 2022+ filter toggle and download button."""
    output_notebook()
    
    country_map = {
        'DZ': 'Algeria', 
        'MA': 'Morocco',
        'US': 'United States'
    }
    
    try:
        df = clean_linkedin_data(excel_file)
        
        if df is None:
            print("Data cleaning failed. Please check your Excel file.")
            return
        
        # Compute global y-axis range for all countries
        global_min = df['LHR (YOY)'].min()
        global_max = df['LHR (YOY)'].max()

        tabs = []

        for idx, (country_code, country_name) in enumerate(country_map.items()):
            country_data = df[df['Country'] == country_name].copy()
            if country_data.empty:
                print(f"No data available for {country_name}.")
                continue

            # Keep datetime format for plotting
            country_data['Month'] = pd.to_datetime(country_data['Month'])
            country_data = country_data.sort_values('Month')

            # Add string version of Month for download
            country_data['Month_str'] = country_data['Month'].dt.strftime('%Y-%m')

            # Prepare sources
            full_data = country_data
            filtered_data = full_data[full_data['Month'] >= '2022-01-01']

            source = ColumnDataSource(full_data)
            source_filtered = ColumnDataSource(filtered_data)
            # Download source: string Month only
            source_download = ColumnDataSource(
                full_data[['Country', 'Month_str', 'LHR (YOY)']].rename(columns={'Month_str': 'Month'})
            )

            # Create plot
            p = figure(
                title=f"LinkedIn Hiring Rate in {country_name}",
                x_axis_type='datetime',
                width=800,
                height=500,
                background_fill_color="#f8f9fa",
                y_range=(global_min, global_max)
            )

            p.line(
                x='Month',
                y='LHR (YOY)',
                source=source,
                line_width=3,
                color=Category20[20][idx % 20]
            )

            hover = HoverTool(tooltips=[
                ("Month", "@Month{%b %Y}"),
                ("LHR (YOY)", "@{LHR (YOY)}{0.00}%")
            ], formatters={"@Month": "datetime"}, mode='vline')
            p.add_tools(hover)

            p.xaxis.axis_label = 'Month'
            p.yaxis.axis_label = 'LinkedIn Hiring Rate (YOY %)'
            p.xaxis.axis_label_text_font_size = '12pt'
            p.yaxis.axis_label_text_font_size = '12pt'
            p.xaxis.major_label_text_font_size = '10pt'
            p.yaxis.major_label_text_font_size = '10pt'
            p.title.text_font_size = '14pt'
            p.grid.grid_line_color = "gray"
            p.grid.grid_line_alpha = 0.3

            # Toggle button
            toggle = Toggle(label="Show only from 2022", button_type="success", active=False)

            callback = CustomJS(args=dict(
                toggle=toggle,
                source=source,
                full=source.data,
                filtered=source_filtered.data
            ), code="""
                source.data = toggle.active ? filtered : full;
                source.change.emit();
                toggle.label = toggle.active ? "Show full range" : "Show only from 2022";
                toggle.button_type = toggle.active ? "warning" : "success";
            """)
            toggle.js_on_change("active", callback)

            # Download button
            download_button = Button(label="Download CSV", button_type="primary")

            download_js = CustomJS(args=dict(source=source_download, name=country_name), code="""
                const data = source.data;
                const cols = ["Country", "Month", "LHR (YOY)"];
                const nrows = data[cols[0]].length;
                let csv = cols.join(",") + "\\n";
                for (let i = 0; i < nrows; i++) {
                    let row = cols.map(col => data[col][i]);
                    csv += row.join(",") + "\\n";
                }
                const blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
                const link = document.createElement("a");
                link.href = URL.createObjectURL(blob);
                const filename = "linkedin_HR_" + name.replace(/ /g, "_") + ".csv";
                link.download = filename;
                link.click();
            """)
            download_button.js_on_click(download_js)

            # Combine all widgetsreate_country_plots
            layout = column(toggle, download_button, p)
            tab = TabPanel(child=layout, title=country_name)
            tabs.append(tab)

        if tabs:
            show(Tabs(tabs=tabs))
        else:
            print("No data available for any of the specified countries.")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check your Excel file structure and column names.")




In [5]:
from bokeh.io import output_notebook, show
output_notebook()
file_path = '../../data/LinkedIn/LinkedIn_LHR by Industry_Feb2025.xlsx'

create_country_plots(file_path)

Loading BokehJS ...

Loading BokehJS ...

### LinkedIn Hiring Rate Insights

#### General Note

To interpret these results appropriately, it's important to consider LinkedIn’s guidance on the level of disaggregation at which the hiring rate estimates are valid:

* For **Algeria, Morocco**, LinkedIn deems the hiring rate valid across **tech-related industries**.

The following insights are presented at the **national level** and are therefore valid for both countries.

#### COVID-19 Impact (2020)

* All markets experienced a **significant contraction** in 2020.
* **Morocco (-0.80%)** and **Algeria (-0.74%)** recorded large declines.

#### Post-COVID Recovery Surge (2021)

* A **sharp increase in April 2021** hiring rates is visible across countries, driven by **year-over-year (YoY) calculations** ([see methodology](https://economicgraph.linkedin.com/content/dam/me/economicgraph/en-us/PDF/linkedin-hiring-rate-methodology.pdf)).

  * This spike is primarily due to **base effects** from April 2020, when hiring plummeted.
  * The surge reflects a **rebound** in labor market activity, not necessarily a hiring boom.

* Peak YoY growth rates in mid-2021:

  * **Morocco:** 3.503% (April)
  * **Algeria:** 3.4545% (April)

* The synchronized peaks suggest a **regional “catch-up” effect** after COVID-19 disruptions.

#### Market Stabilization Patterns (2022–2024)

* **Morocco**: Continued growth through 2022 until **December**, followed by mostly negative or near-zero growth.
* **Algeria**: Stable trend around zero; **slightly negative** since **December 2023**.



In [6]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Tabs, TabPanel, Div
from bokeh.layouts import column

def plot_epr_growth_tabs(employment_pivot):
    """
    Create Bokeh tabs (one per country) plotting EPR_Growth_pct vs Year.
    Expects a DataFrame with columns: 'Country Name', 'Year', 'EPR_Growth_pct'.
    Includes an additional README tab describing data source and computation.
    """

    output_notebook()
    tabs = []

    # Ensure numeric year
    employment_pivot = employment_pivot.copy()
    employment_pivot['Year'] = pd.to_numeric(employment_pivot['Year'], errors='coerce')

    # Get list of countries dynamically
    countries = employment_pivot['Country Name'].dropna().unique()

    for country in countries:
        dfc = employment_pivot[employment_pivot['Country Name'] == country].dropna(subset=['EPR_Growth_pct'])
        if dfc.empty:
            continue

        source = ColumnDataSource(dfc)

        p = figure(
            title=f"{country} — Employment-to-Population Growth (YoY %)",
            width=850,
            height=400,
            background_fill_color="#f8f9fa"
        )

        p.line('Year', 'EPR_Growth_pct', source=source, line_width=3, color="#4a6fa5")
        p.scatter('Year', 'EPR_Growth_pct', source=source, size=7, color="#2ca02c", line_color="white", line_width=1.5)

        # Add hover tool
        hover = HoverTool(tooltips=[
            ("Year", "@Year"),
            ("EPR Growth (YoY %)", "@EPR_Growth_pct{0.00}"),
        ])
        p.add_tools(hover)

        # Add axis labels and grid
        p.xaxis.axis_label = "Year"
        p.yaxis.axis_label = "EPR Growth (YoY %)"
        p.grid.grid_line_color = "gray"
        p.grid.grid_line_alpha = 0.3
        p.title.text_font_size = "14pt"

        tabs.append(TabPanel(child=column(p), title=country))

    # Add README tab
    readme_html = """
    <h3>About this visualization</h3>
    <p>This figure presents the <b>year-to-year growth</b> in the
    <i>Employment-to-Population Ratio</i> for each selected country.</p>

    <h4>Computation</h4>
    <p>
    The indicator plotted here (<b>EPR_Growth_pct</b>) is calculated as the
    year-over-year percentage change in the employment-to-population ratio:
    </p>
    <pre>
    EPR_Growth_pct = ((EPR<sub>t</sub> - EPR<sub>t-1</sub>) / EPR<sub>t-1</sub>) × 100
    </pre>

    <h4>Interpretation</h4>
    <p>
    Positive values indicate an increase in the share of employed people relative to the working-age population compared to the previous year.
    Negative values indicate a decline.
    </p>
    
    <h4>Data Source</h4>
    <p>
    World Bank – <a href="https://data.worldbank.org/indicator/SL.EMP.TOTL.SP.ZS?locations=DZ" target="_blank">
    Employment to population ratio, 15+, total (%) (modeled ILO estimate)</a>.
    </p>
    """

    readme_tab = TabPanel(child=Div(text=readme_html, width=850), title="README")
    tabs.append(readme_tab)

    # Display all tabs
    if tabs:
        show(Tabs(tabs=tabs))
    else:
        print("No data available for the selected countries.")


plot_epr_growth_tabs(employment_pivot)


Loading BokehJS ...

> The trend observed in the World Development Indicators (WDI) data closely mirrors the pattern captured by the LinkedIn Hiring Rate (LHR). Both datasets show a **decline in 2020**, followed by a **strong recovery in 2021 and 2022**, and then a period of **stagnation in 2023** with a **slight decline in 2024**.

> This alignment suggests that the **LinkedIn hiring data reflects broader labor market dynamics in Algeria**, indicating that online hiring activity captured by LinkedIn can serve as a proxy for official employment trends in the general economy.
